# ANÁLISIS DEL DATASET "DIAMONDS".

## 1-CARGA DE DATOS, LIBRERÍAS DE PYTHON Y FUNCIONES AUXILIARES.

### 1.1-CARGA DE LIBRERÍAS DE PYTHON.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px

### 1.2-CARGA DE DATOS.

In [2]:
# Archivo fuente de datos
file = './Data/diamonds.csv'

# Carga del archivo fuente y generación del DataFrame
df = pd.read_csv(file)

### 1.3-FUNCIONES AUXILIARES.

In [49]:
# Función para reemplazar valores 'NaN': por la mediana, en el caso de valores numéricos; por la moda, en el caso de valores categóricos.
# Entrada: pd.DataFrame
# Salida: pd.DataFrame / Se imprime en pantalla el número de reemplazos realizados por la moda y por la mediana.
def replace_nan(df):
    contador_modas = 0
    contador_medianas = 0
    for columna in df.columns:
        if df[columna].dtype == 'object' | df[columna].dtype == 'boolean': # Los valores son de tipo texto ('object') o boleanos.
            df[columna].fillna(df[columna].mode()[0], inplace=True)
            contador_modas += 1
        elif df[columna].dtype in ['float8', 'float16', 'float32', 'float64', 'int8', 'int16', 'int32', 'int64']: # Los valores son de tipo numérico. 
            df[columna].fillna(df[columna].median(), inplace=True)
            contador_medianas += 1
    if contador_modas == 0 & contador_medianas == 0: # No se han encontrado valores NaN en el DataFrame.
        print('No se han encontrado valores NaN que sustituir.')
    elif contador_modas > 0: # Se han reemplazado valores NaN por modas.
        print(f'Se han relizado {contador_modas} reemplazos por la moda de la columna.')
    elif contador_medianas > 0: # Se han reemplazado valores NaN por medianas.
        print(f'Se han relizado {contador_medianas} reemplazos por la mediana de la columna.')
    else:
        None
    return df

# Función para buscar coincidencias de caracteres en las columnas de un DataFrame.
# Entrada: pd.DataFrame, string
# Salida: Lista con los nombbres de las columnas del DataFrame donde se encontraron coincidencias.
def coincidencias_str_df (df, caracter):
    lista_coincidencias = [] # Lista para almacenar los nombres de las columnas donde se encontraron coincidencias.
    for columna in df.columns:
       if df[columna].astype(str).str.contains(caracter).any(): # Se convierten los valores de la columna a string y se busca el caracter en la columna.
           lista_coincidencias.append(columna)
    return lista_coincidencias



## 2-ANÁLISIS PRELIMINAR DEL DATAFRAME.

### 2.1-ANÁLISIS DE LA ESTRUCTURA DEL DATAFRAME.

#### 2.1.1-VISUALIZACIÓN PREVIA DEL DATAFRAME.

In [3]:
df.head(5)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.2,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
df[25000::5]

,carat,cut,color,clarity,depth,table,price,x,y,z
25000,1.29,Ideal,E,VVS2,59.8,59.0,13530,7.07,7.11,4.24
25005,1.50,Very Good,F,VS2,59.7,56.0,13537,7.42,7.46,4.44
25010,2.00,Good,J,VS2,61.4,63.0,13542,8.01,8.08,4.94
25015,1.50,Fair,G,VVS2,64.8,56.0,13553,7.18,7.12,4.63
25020,0.38,Ideal,F,SI1,61.8,55.0,640,4.66,4.69,2.89
...,...,...,...,...,...,...,...,...,...,...
53915,0.77,Ideal,E,SI2,62.1,56.0,2753,5.84,5.86,3.63
53920,0.70,Very Good,E,VS2,62.4,60.0,2755,5.57,5.61,3.49
53925,0.79,Ideal,I,SI1,61.6,56.0,2756,5.95,5.97,3.67
53930,0.71,Premium,E,SI1,60.5,55.0,2756,5.79,5.74,3.49


In [5]:
df.tail(5)

,carat,cut,color,clarity,depth,table,price,x,y,z
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74
53939,0.75,Ideal,D,SI2,62.2,55.0,2757,5.83,5.87,3.64


Como se puede observar en las tablas superiores, el DataFrame cargado consta de 53.940 filas y 10 columnas, de las cuales, 7 columnas son numéricas y 3 columnas son categóricas, en principio.

### 2.2-ANÁLISIS DE LA INCONSISTENCIA DE DATOS.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53939 non-null  float64
 1   cut      53937 non-null  object 
 2   color    53938 non-null  object 
 3   clarity  53938 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53938 non-null  float64
 6   price    53938 non-null  object 
 7   x        53938 non-null  object 
 8   y        53939 non-null  float64
 9   z        53939 non-null  float64
dtypes: float64(5), object(5)
memory usage: 4.1+ MB


Al ampliar la información del DataFrame, como se puede apreciar en la tabla superior, se indica que el DataFrame tiene 53.940 filas. Sin embargo, existe una única columna (la 4 - 'depth') en cuyas observaciones no existe ningún valor nulo. En el resto de columnas, se han detectado valores nulos: existen columnas con 1 valor nulo ('carat', 'y', 'z'), 2 valores nulos ('color', 'clarity', 'table' y 'price') y 3 valores nulos ('cut').
<p>Por otra parte, algunas columnas no tienen el tipo de dato que se esperaba a priori. Por ejemplo, las columnas 'price' y 'x', en principio, se esperaban que contuvieran valores numéricos y se han detectado valores de tipo 'object', por lo que los valores de las columnas indicadas pueden haberse cargado como tipo 'string'.
<P>Por todo ello, se deduce que existen valores inconsistentes en las columnas mencionadas que hay que tratar adecuadamente. 
<p>En principio, las cantidades de estas inconsitencias encontradas no son muy numerosas. Dada la cantidad total de las filas del DataFrame, las inconsistencias no parecen tener un peso importante en el conjunto del mismo, y una de las opciones podría ser eliminar directamente las filas afectadas. No obstante, es preferible evitar su eliminación (afín de no perder información) y las filas/columnas afectadas se van a tratar adecuadamente. Para ello, a continuación se procede al análisis de cada columna afectada.

### 2.3-ANÁLISIS POR COLUMNAS.

#### 2.3.1-COLUMNA: 'carat'.

In [7]:
# Se imprimen los valores únicos de la columna. Se aprecia que el último valor obtenido es de tipo 'NaN
print(df['carat'].unique())

[0.23 0.21 0.29 0.31 0.24 0.26 0.22 0.3  0.2  0.32 0.33 0.25 0.35 0.42
 0.28 0.38 0.7  0.86 0.71 0.78 0.96 0.73 0.8  0.75 0.74 0.81 0.59 0.9
 0.91 0.61 0.77 0.63 0.76 0.64 0.72 0.79 0.58 1.17 0.6  0.83 0.54 0.98
 0.52 1.01 0.53 0.84 0.51 1.05 0.55 0.87 1.   0.57 0.82 1.04 0.93 1.2
 0.99 0.34 0.43 0.36 0.95 0.89 1.02 0.97 0.56 0.85 0.92 1.27 0.66 1.12
 0.68 1.03 0.62 1.22 1.08 0.88 0.5  1.19 0.39 0.65 1.24 1.5  0.27 0.41
 1.13 1.06 0.69 0.4  1.14 0.94 1.29 1.52 1.16 1.21 1.23 1.09 0.67 1.11
 1.1  1.18 1.15 1.25 1.07 1.28 1.51 0.37 1.31 1.26 1.39 1.44 1.35 1.3
 1.32 1.41 1.36 1.45 1.34 1.58 1.54 1.38 1.33 1.74 1.64 1.47 1.4  1.55
 1.95 2.   1.37 1.83 1.62 1.57 1.69 2.06 1.72 1.66 2.14 1.49 1.46 2.15
 1.96 2.22 1.7  1.53 1.85 2.01 2.27 1.68 1.56 1.81 1.65 1.82 2.03 1.73
 1.59 1.42 1.43 2.08 1.48 1.6  2.49 1.71 2.02 2.07 3.   2.21 2.1  1.91
 2.25 2.17 2.32 2.72 1.61 2.23 2.11 2.05 1.63 2.3  2.31 1.75 2.04 2.12
 1.77 2.5  1.8  1.67 1.84 2.2  3.01 1.88 2.33 2.68 2.34 1.9  2.16 2.74
 1.78 1.7

In [10]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['carat'].value_counts(dropna=False, sort=True, ascending=True)

carat
NaN        1
3.40       1
3.67       1
4.00       1
3.50       1
        ... 
0.32    1840
0.70    1981
1.01    2242
0.31    2249
0.30    2604
Name: count, Length: 274, dtype: int64

A parte de los valores NaN, no se aprecia ningún otro valor inconsistente en la columna.

In [11]:
# Se imprime la línea que contiene el valor 'NaN' en la columna 'carat'
print(df[df['carat'].isnull()])

       carat      cut color clarity  depth  table price     x     y     z
37626    NaN  Premium     G     SI1   61.0   60.0   990  4.93  4.88  2.99


In [16]:
filas_valor_cero = df[df['carat']==0]
filas_valor_cero

,carat,cut,color,clarity,depth,table,price,x,y,z


#### 2.3.2-COLUMNA: 'cut'.

In [12]:
# Se imprimen los valores únicos de la columna.
print(df['cut'].unique())

['Ideal' 'Premium' 'Good' 'Very Good' 'Fair' nan '?']


In [13]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['cut'].value_counts(dropna=False)

cut
Ideal        21549
Premium      13791
Very Good    12080
Good          4906
Fair          1610
NaN              3
?                1
Name: count, dtype: int64

Se observa que, además de los 3 valores NaN, también aparece un valor '?', que no se correspondería con un valor esperado en la columna 'cut'.

In [14]:
# Se imprime la fila que contiene el caracter '?' en la columna 'cut'
df[df['cut'].str.contains(r'\?', na=False)]

,carat,cut,color,clarity,depth,table,price,x,y,z
49227,0.5,?,E,VVS2,61.7,61.0,2083,5.09,5.12,3.15


In [17]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna 'cut'
print(df[df['cut'].isnull()])


       carat  cut color clarity  depth  table  price     x     y     z
14432   1.20  NaN     G     SI2   61.1   57.0   5826  6.86  6.83  4.18
14468   1.08  NaN     H     VS2   62.1   56.0   5839  6.59  6.56  4.08
27259   2.01  NaN     E     SI1   62.7   57.0  17676  8.03  8.11  5.06


In [19]:
filas_valor_cero = df[df['cut'].str.startswith('0', na=False)]
filas_valor_cero

,carat,cut,color,clarity,depth,table,price,x,y,z


No hay valores '0'.

#### 2.3.3-COLUMNA: 'color'.

In [20]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
print(df['color'].value_counts(dropna=False))

color
G      11291
E       9797
F       9542
H       8304
D       6774
I       5421
J       2808
NaN        2
?          1
Name: count, dtype: int64


Se observa que, además de los 2 valores 'NaN', también aparece un valor '?' no esperado en la columna.

In [22]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna 'color'
print(df[df['color'].isnull()])


       carat    cut color clarity  depth  table  price     x     y     z
27260   2.32  Ideal   NaN     SI2   62.0   57.0  17676  8.49  8.45  5.25
37629   0.40  Ideal   NaN     SI1   61.8   55.0    990  4.78  4.74  2.94


In [21]:
filas_valor_cero = df[df['color'].str.startswith('0', na=False)]
filas_valor_cero

,carat,cut,color,clarity,depth,table,price,x,y,z


#### 2.3.4-COLUMNA: 'clarity'.

In [23]:
# Se imprimen los valores únicos de la columna.
print(df['clarity'].unique())

['SI2' 'SI1' 'VS1' 'VS2' 'VVS2' 'VVS1' 'I1' 'IF' nan]


In [24]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['clarity'].value_counts(dropna=False)

clarity
SI1     13065
VS2     12257
SI2      9194
VS1      8170
VVS2     5066
VVS1     3655
IF       1790
I1        741
NaN         2
Name: count, dtype: int64

En este caso, aparecen 2 valores NaN.

In [ ]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna 'clarity'
print(df[df['clarity'].isnull()])


In [25]:
filas_valor_cero = df[df['clarity'].str.startswith('0', na=False)]
filas_valor_cero

,carat,cut,color,clarity,depth,table,price,x,y,z


#### 2.3.5-COLUMNA: 'table'.

In [26]:
# Se imprimen los valores únicos de la columna.
print(df['table'].unique())

[55.  61.  65.  58.  57.  56.  54.  62.  59.  63.  60.  55.3 69.  53.
 64.  56.4 60.5 67.  52.  55.8 66.  70.  68.  54.4 57.1 53.6 54.8 54.2
 54.9 56.7 53.9 57.8 54.1 55.1 51.  58.1 56.9 56.5 53.7 60.7 56.2 56.6
 56.1 56.3 57.2 57.5 60.9 59.8 62.3 60.1 55.9 50.1 62.2 54.5 55.7 53.4
 54.7 58.8 59.4 62.5 54.3 60.6 55.2 58.6 49.  50.  54.6 62.6 57.7 56.8
 57.9 60.3 55.6 59.5 57.4 62.4 60.4 53.3 60.2 60.8 58.5 57.6 43.  59.9
 63.5 57.3 61.4 58.3 55.4 58.2 53.5 59.7  nan 55.5 58.7 53.8 59.1 53.2
 59.3 61.1 51.6 95.  61.2 59.2 58.4 53.1 52.8 44.  61.3 61.9 61.8 59.6
 61.5 58.9 64.3 62.8 71.  52.4 63.3 61.7 64.2 73.  65.4 63.4 62.1 79.
 76.  61.6]


In [27]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['table'].value_counts(dropna=False)

table
56.0    9881
57.0    9724
58.0    8368
59.0    6571
55.0    6268
        ... 
63.4       1
62.1       1
79.0       1
76.0       1
61.6       1
Name: count, Length: 128, dtype: int64

In [28]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna.
print(df[df['table'].isnull()])


       carat      cut color clarity  depth  table  price     x     y     z
14445   1.22  Premium     I     VS2   62.3    NaN   5832  6.79  6.83  4.24
27256   2.28  Premium     I     SI1   61.6    NaN  17673  8.43  8.46  5.20


Aquí aparecen otros 2 valores NaN.

In [29]:
filas_valor_cero = df[df['table']==0]
filas_valor_cero

,carat,cut,color,clarity,depth,table,price,x,y,z


#### 2.3.6-COLUMNA: 'price'.

In [30]:
# Se imprimen los valores únicos de la columna.
print(df['price'].unique())

['326' '327' '334' ... '2753' '2755' '2756']


In [31]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['price'].value_counts(dropna=False)

price
605     132
802     127
625     126
828     125
776     124
       ... 
9043      1
9044      1
2783      1
2794      1
2796      1
Name: count, Length: 11603, dtype: int64

In [32]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna.
print(df[df['price'].isnull()])


       carat        cut color clarity  depth  table price     x     y     z
14459   1.24  Very Good     I     VS2   62.7   55.0   NaN  6.82  6.87  4.29
27249   2.02  Very Good     F     SI1   60.1   60.0   NaN  8.21   NaN  4.96


Esta columna 'price' contiene valores que representan precios (en formato string) de diamantes. Dado que un precio '0' puede representar un valor anómalo no esperado, se busca en la columna si existe algún valor '0' o '0.'. Dando como resultado que no existe ningún valor de este tipo. En un apartado posterior, se realizará la conversión de tipo de dato a un tipo numérico más adecuado.

In [33]:
print(('0' in df['price']) | ('0.' in df['price']))


False


#### 2.3.7-COLUMNA: 'x'.

In [34]:
# Se imprimen los valores únicos de la columna.
print(df['x'].unique())

['3.95' '3.89' '4.05' '4.2' '4.34' '3.94' '4.07' '3.87' '4' '4.25' '3.93'
 '3.88' '4.35' '3.79' '4.38' '4.31' '4.23' '4.21' '4.26' '3.85' '4.39'
 '4.44' '3.97' '4.28' '3.96' '4.04' '4.01' '3.92' '4.06' '3.83' '4.29'
 '4.13' '4.49' '3.99' '4.19' '4.24' '4.36' '4.33' '4.02' '4.45' '3.91'
 '4.3' '4.43' '4.54' '4.78' '4.15' '4.08' '4.03' '4.65' '4.22' '5.7'
 '6.45' '5.68' '5.81' '5.85' '5.71' '6.27' '5.77' '5.97' '5.8' '6' '5.94'
 '5.87' '5.9' '6.14' '5.38' '5.96' '6.16' '5.73' '5.98' '5.72' '6.01'
 '5.64' '5.57' '5.63' '5.65' '6.09' '5.56' '6.11' '6.03' '5.89' '5.69'
 '5.62' '5.88' '5.52' '5.74' '5.53' '5.78' '5.83' '5.66' '5.82' '5.79'
 '5.92' '5.39' '5.33' '6.83' '5.41' '5.25' '5.24' '5.67' '5.61' '5.76'
 '6.05' '5.6' '5.19' '5.84' '6.39' '5.34' '5.14' '5.11' '5.54' '6.07'
 '6.29' '6.31' '6.49' '5.28' '5.48' '5.55' '6.41' '6.02' '5.91' '6.17'
 '6.61' '5.12' '6.19' '5.43' '5.99' '5.95' '6.37' '5.93' '6.46' '5.22'
 '5.75' '6.21' '6.15' '6.73' '5.49' '4.37' '4.47' '4.41' '4.27' '4.85'
 '4.

In [35]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['x'].value_counts(dropna=False)

x
4.37    448
4.34    437
4.33    429
4.38    428
4.32    425
       ... 
9.49      1
9.65      1
9.53      1
8.79      1
?         1
Name: count, Length: 556, dtype: int64

Aparece un valor '?'.

In [36]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna.
print(df[df['x'].isnull()])


       carat    cut color clarity  depth  table price    x     y     z
14446   1.01  Ideal     D     SI1   62.0   57.0  5832  NaN  6.44  3.97
14470   1.00   Fair     F     VS2   64.9   56.0  5840  NaN  6.31  4.08


In [37]:
filas_valor_cero = df[df['x'].str.startswith('0', na=False)]
filas_valor_cero

,carat,cut,color,clarity,depth,table,price,x,y,z
11182,1.07,Ideal,F,SI2,61.6,56.0,4954,0,6.62,0.0
11963,1.00,Very Good,H,VS2,63.3,53.0,5139,0,0.00,0.0
15951,1.14,Fair,G,VS1,57.5,67.0,6381,0,0.00,0.0
24520,1.56,Ideal,G,VS2,62.2,54.0,12800,0,0.00,0.0
26243,1.20,Premium,D,VVS1,62.1,59.0,15686,0,0.00,0.0
27429,2.25,Premium,H,SI2,62.8,59.0,18034,0,0.00,0.0
49556,0.71,Good,F,SI2,64.1,60.0,2130,0,0.00,0.0
49557,0.71,Good,F,SI2,64.1,60.0,2130,0,0.00,0.0


En esta columna, también aparecen varios valores '0', que deberán ser tratados adecuadamente.

#### 2.3.8-COLUMNA: 'y'.

In [38]:
# Se imprimen los valores únicos de la columna.
print(df['y'].unique())

[ 3.98  3.84  4.07  4.23  4.35  3.96  4.11  3.78  4.05  4.28  3.9   4.37
  3.75  4.42  4.34  4.29  4.26  4.27  4.3   3.92  4.43  4.47  4.01  3.94
  3.97  3.99  4.03  4.06  4.08  3.85  4.31  4.16  4.51  4.5   4.55  4.02
  4.24  4.38  3.95  4.48  3.89  3.88  4.33  4.32  4.59  4.84  4.22  4.
  4.25  4.13  4.09  4.04  4.12  4.67  5.72  6.33  5.67  5.73  5.85  5.9
  5.76  5.95  5.78  5.93  5.75  5.96  5.79  5.87  5.8   5.81  6.11  5.43
  6.    5.84  6.12  5.82  5.74  5.94  6.07  5.69  5.83  5.61  5.53  5.58
  5.59  5.97  6.09  5.99  5.86  5.68  5.65  5.55  5.77  5.91  5.56  5.89
  5.92  6.03  5.44  5.37  5.88  6.9   5.62  5.27  5.26  5.7   5.64  5.63
  5.71  5.51  5.22  6.36  5.54  5.66  6.2   5.18  5.15  6.1   6.21  6.22
  6.45  6.01  5.3   5.41  5.52  6.27  6.05  6.06  6.13  6.55  5.35  5.16
  6.25  6.02  5.46  6.23  6.08  5.45  6.41  6.34  5.25  6.15  6.19  6.66
  4.36  4.2   4.79  4.57  4.52  4.53  4.58  5.    6.67  5.11  6.29  6.04
  6.53  6.65  6.5   6.16  5.6   6.39  5.21  5.4   5.98

In [39]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['y'].value_counts(dropna=False)

y
4.34     437
4.37     435
4.35     425
4.33     421
4.32     414
        ... 
9.03       1
9.85       1
3.81       1
3.82       1
31.80      1
Name: count, Length: 553, dtype: int64

In [40]:
filas_valor_cero = df[df['y']==0]
filas_valor_cero

,carat,cut,color,clarity,depth,table,price,x,y,z
11963,1.00,Very Good,H,VS2,63.3,53.0,5139,0,0.0,0.0
15951,1.14,Fair,G,VS1,57.5,67.0,6381,0,0.0,0.0
24520,1.56,Ideal,G,VS2,62.2,54.0,12800,0,0.0,0.0
26243,1.20,Premium,D,VVS1,62.1,59.0,15686,0,0.0,0.0
27429,2.25,Premium,H,SI2,62.8,59.0,18034,0,0.0,0.0
49556,0.71,Good,F,SI2,64.1,60.0,2130,0,0.0,0.0
49557,0.71,Good,F,SI2,64.1,60.0,2130,0,0.0,0.0


In [41]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna.
print(df[df['y'].isnull()])


       carat        cut color clarity  depth  table price     x   y     z
27249   2.02  Very Good     F     SI1   60.1   60.0   NaN  8.21 NaN  4.96


#### 2.3.9-COLUMNA: 'z'.

In [42]:
# Se imprimen los valores únicos de la columna.
print(df['z'].unique())

[ 2.43  2.31  2.63  2.75  2.48  2.47  2.53  2.49  2.39  2.73  2.46  2.33
  2.71  2.27  2.68  2.7   2.66  2.41  2.62  2.59  2.67  2.4   2.42  2.44
  2.37  2.52  2.78  2.76  2.61  2.65  2.79  2.54  2.64  2.45  2.72  2.69
  2.96  2.58  2.51  2.56  2.55  2.87  3.57  3.52  3.5   3.56  3.72  3.38
  3.4   4.07  3.66  3.65  3.58  3.59  3.63  3.69  3.61  3.6   3.35  3.67
  3.62  3.87  3.68  3.49  3.47  3.46  3.7   3.32  3.93  3.95  3.64  3.53
  3.37  3.43  3.51  3.54  3.48  3.55  3.33  3.3   4.13  3.79  3.24  3.23
  4.08  3.19  3.94  3.08  3.31  3.21  3.75  3.18  3.41  4.03  4.09  3.22
  4.18  3.76  3.83  3.17  3.44  3.74  3.8   3.39  3.88  4.    3.98  3.85
  3.96  3.16  3.78  3.84  4.33  4.01  3.45  2.74  2.95  2.8   2.77  2.9
  2.82  3.77  3.9   3.73  3.34  3.86  3.89  3.81  3.42  3.71  3.91  3.2
  3.1   4.06  3.97  4.02  4.2   4.11  3.99  4.23  3.27  3.92  4.15  2.83
  2.81  4.04  3.82  4.21  4.1   4.26  4.29  3.26  2.88  2.5   4.16  4.14
  4.12  3.25  4.7   4.19  3.13  4.05  4.27  4.34  3.1

In [43]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['z'].value_counts(dropna=False)

z
2.70     767
2.69     748
2.71     738
2.68     730
2.72     697
        ... 
5.75       1
2.29       1
2.06       1
2.25       1
31.80      1
Name: count, Length: 376, dtype: int64

In [44]:
filas_valor_cero = df[df['z']==0]
filas_valor_cero

,carat,cut,color,clarity,depth,table,price,x,y,z
2207,1.00,Premium,G,SI2,59.1,59.0,3142,6.55,6.48,0.0
2314,1.01,Premium,H,I1,58.1,59.0,3167,6.66,6.60,0.0
4791,1.10,Premium,G,SI2,63.0,59.0,3696,6.5,6.47,0.0
5471,1.01,Premium,F,SI2,59.2,58.0,3837,6.5,6.47,0.0
10167,1.50,Good,G,I1,64.0,61.0,4731,7.15,7.04,0.0
11182,1.07,Ideal,F,SI2,61.6,56.0,4954,0,6.62,0.0
11963,1.00,Very Good,H,VS2,63.3,53.0,5139,0,0.00,0.0
13601,1.15,Ideal,G,VS2,59.2,56.0,5564,6.88,6.83,0.0
15951,1.14,Fair,G,VS1,57.5,67.0,6381,0,0.00,0.0
24394,2.18,Premium,H,SI2,59.4,61.0,12631,8.49,8.45,0.0


In [45]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna.
print(df[df['z'].isnull()])


       carat      cut color clarity  depth  table price     x     y   z
14472   1.23  Premium     F     SI2   59.9   58.0  5841  7.02  6.96 NaN


#### 2.3.10-COLUMNA: 'depth'.

In [46]:
# Se imprimen los valores únicos de la columna.
print(df['depth'].unique())

[61.5 59.8 56.9 62.4 63.3 62.8 62.3 61.9 65.1 59.4 64.  60.4 62.2 60.2
 60.9 62.  63.4 63.8 62.7 61.  58.1 62.5 60.5 60.  60.7 59.5 58.2 64.1
 60.8 61.8 61.2 61.1 65.2 58.4 63.1 61.6 59.3 62.6 63.  63.2 62.1 61.4
 62.9 63.7 59.2 59.9 57.9 55.1 57.5 66.3 61.7 58.8 64.5 65.3 59.6 64.4
 65.7 63.6 61.3 60.1 60.3 58.  64.6 59.7 57.8 67.9 60.6 57.2 64.2 65.8
 67.4 59.  63.5 67.3 58.7 66.4 68.1 63.9 55.  58.6 64.3 58.5 65.  56.
 58.3 53.1 64.9 59.1 58.9 66.7 57.7 65.4 53.3 53.  67.8 66.1 55.8 67.6
 68.2 65.5 67.7 69.5 56.6 56.3 66.9 66.  67.  57.6 67.1 65.6 64.8 69.3
 66.2 55.4 66.8 64.7 66.6 55.9 57.3 57.4 68.3 68.5 56.2 65.9 56.5 56.1
 66.5 68.4 69.7 57.1 68.7 56.7 68.6 71.6 43.  68.8 67.5 69.  55.2 68.9
 69.6 57.  56.4 56.8 44.  67.2 70.1 71.3 70.6 69.8 71.8 53.8 53.2 70.
 69.4 68.  70.2 50.8 55.6 70.5 71.  69.1 55.3 54.2 51.  70.8 54.6 54.
 54.4 52.3 55.5 78.2 71.2 52.7 54.3 69.2 73.6 52.2 69.9 53.4 72.2 79.
 72.9 54.7]


In [47]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['depth'].value_counts(dropna=False)

depth
62.0    2239
61.9    2163
61.8    2077
62.2    2039
62.1    2020
        ... 
52.2       1
53.4       1
72.2       1
72.9       1
54.7       1
Name: count, Length: 184, dtype: int64

In [48]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna.
print(df[df['depth'].isnull()])


Empty DataFrame
Columns: [carat, cut, color, clarity, depth, table, price, x, y, z]
Index: []


La columna 'depth' no presenta ni valores 'NaN' ni ningún otro tipo de dato inconsistente.

### 2.4-LIMPIEZA Y TRATAMIENTO DE DATOS NULOS O INCONSISTENTES.

Resúmen de las inconsistencias detectadas.

<table style="width:20%">
<tr>
<th></th>
<th>CARAT</th>
<th>CUT</th>
<th>COLOR</th>
<th>CLARITY</th>
<th>DEPTH</th>
<th>TABLE</th>
<th>PRICE</th>
<th>X</th>
<th>Y</th>
<th>Z</th>
</tr>
<tr>
<td>TIPO DATO</td>
<td>FLOAT64</td>
<td>OBJECT</td>
<td>OBJECT</td>
<td>OBJECT</td>
<td>FLOAT64</td>
<td>FLOAT64</td>
<td>OBJECT</td>
<td>OBJECT</td>
<td>FLOAT64</td>
<td>FLOAT64</td>
</tr>

<tr>
<td>NaN</td>
<td>SÍ</td>
<td>SÍ</td>
<td>SÍ</td>
<td>SÍ</td>
<td>NO</td>
<td>SÍ</td>
<td>SÍ</td>
<td>SÍ</td>
<td>SÍ</td>
<td>SÍ</td>
</tr>

<tr>
<td>?</td>
<td>NO</td>
<td>SÍ</td>
<td>SÍ</td>
<td>NO</td>
<td>NO</td>
<td>NO</td>
<td>NO</td>
<td>SÍ</td>
<td>NO</td>
<td>NO</td>
</tr>

<tr>
<td>0</td>
<td>NO</td>
<td>NO</td>
<td>NO</td>
<td>NO</td>
<td>NO</td>
<td>NO</td>
<td>NO</td>
<td>SÍ</td>
<td>SÍ</td>
<td>SÍ</td>
</tr>
</table>


De los datos inconsistentes detectados, tal y como aparecen en la tabla superior, el proceso que se va a realizar es el siguiente:
<ol>
<li>Sustituir los valores '?' por otro valor, por ejemplo 'NaN', para que se permita realizar adecuadamente la conversión de tipos de datos.</li>
<li>Sustituir los valores '0' y '0.' por otro valor, por ejemplo 'NaN', para sustituir posteriormente dichos valores por la mediana.</li>
<li>Realizar la conversión de tipo de datos.</li>
<li>Sustituir los valores 'NaN' por las medianas (valores numéricos) o por las modas (valores categóricos).</li>
</ol>

Se va a sustituir el valor 'NaN' por la mediana de la columna.

In [ ]:
# Se obtienen las columnas con, al menos, un valor NaN
columnas_nan = df.columns[df.isnull().any()]
print("Columnas con valores NaN:", columnas_nan)

# Reemplazar NaN con la mediana para columnas numéricas y la moda para categóricas
for columna in columnas_nan:
    if df[columna].dtype in ['float64', 'int64']:  # Si la columna es numérica
        median = df[columna].median()
        df[columna].fillna(median, inplace=True) # Se reemplaza NaN por la mediana
    else:  # Si la columna es categórica o string ('object')
        mode = df[columna].mode().iloc[0]  # Se selecciona la primera moda
        df[columna].fillna(mode, inplace=True) # Se reemplaza NaN por la moda

# Resultado
""" print("\nDataFrame después de reemplazar valores NaN:")
print(df) """
df.info()


In [ ]:
# Se reemplazan los carácteres '?' con NaN en la columna
df['cut'] = df['cut'].replace('?', np.nan)
df.replace('?', np.nan)